# Transform NY dataset into dataframe

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [8]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# Tranform Toronto dataset into dataframe

In [9]:
import requests
import lxml.html as lh

In [10]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')


#Store the contents of the website under doc
doc = lh.fromstring(website_url.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    name=t.text_content().rstrip()
    col.append((name,[]))
    
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=str(data).rstrip()
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [11]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [12]:
df = df[df.Borough != 'Not assigned']

In [13]:
df.loc[df['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = df['Borough']

In [14]:
data = df.groupby(['Postcode', 'Borough'], as_index = False).agg({'Neighbourhood': ', '.join})

In [15]:
# @hidden_cell
CLIENT_ID = 'ZQ20W2PK1HYQ0VLCTNKJCTRCTUSMAUSOLCSQOJXSYKPED0A0' # your Foursquare ID
CLIENT_SECRET = 'H5XKZHW5L5IHMQ1NH0QTUFMIVIVFEEGDVYMCZFSTCH030NP2' # your Foursquare Secret
VERSION = '20200210'
LIMIT = 30

In [16]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

# All requested packages already installed.



In [17]:
!wget -O coordinate.csv http://cocl.us/Geospatial_data

--2020-02-19 13:49:16--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.83, 158.85.108.86, 169.48.113.194
Connecting to cocl.us (cocl.us)|158.85.108.83|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-02-19 13:49:16--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|158.85.108.83|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-19 13:49:18--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-19 13:49:18--  https://ib

In [18]:
coor = pd.read_csv('coordinate.csv')

In [19]:
data['Latitude'] = coor['Latitude']
data['Longitude'] = coor['Longitude']

In [20]:
address = 'Toronto, TR'

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6561136, -79.392321.


In [21]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighbourhood']):
    label = '{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Explore Neighborhoods in NY & Toronto

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [23]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],latitudes=neighborhoods['Latitude'],longitudes=neighborhoods['Longitude'])
toronto_venues = getNearbyVenues(names=data['Neighbourhood'],latitudes=data['Latitude'],longitudes=data['Longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [24]:
newyork_venues.groupby('Neighbourhood').count()
print('There are {} uniques categories.'.format(len(newyork_venues['Venue Category'].unique())))

There are 378 uniques categories.


In [25]:
toronto_venues.groupby('Neighbourhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 226 uniques categories.


# Cluster Neighborhoods for NY and Toronto

In [26]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
newyork_onehot['Neighbourhood'] = newyork_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [27]:
newyork_grouped = newyork_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [28]:
# set number of clusters
kclusters = 5

newyork_grouped_clustering = newyork_grouped.drop('Neighbourhood', 1)
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
nykmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)
trkmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [29]:
trkmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nyneighborhoods_venues_sorted = pd.DataFrame(columns=columns)
trneighborhoods_venues_sorted = pd.DataFrame(columns=columns)
nyneighborhoods_venues_sorted['Neighbourhood'] = newyork_grouped['Neighbourhood']
trneighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(newyork_grouped.shape[0]):
    nyneighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

for ind in np.arange(toronto_grouped.shape[0]):
    trneighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [32]:
# add clustering labels
nyneighborhoods_venues_sorted.insert(0, 'Cluster Labels', nykmeans.labels_)
trneighborhoods_venues_sorted.insert(0, 'Cluster Labels', trkmeans.labels_)

In [38]:
newyork_merged = neighborhoods
toronto_merged = data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(nyneighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')
toronto_merged = toronto_merged.join(trneighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how = 'right')

In [70]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
ny_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(ny_clusters)
       
ny_clusters

In [69]:
address = 'Toronto, TR'

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
tr_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(tr_clusters)
       
tr_clusters

# Compare clusters between two cities

In [40]:
from sklearn import metrics

## Extract clusters in NY

In [55]:
nycluster1 = newyork_merged.loc[newyork_merged['Cluster Labels'] == 0].drop(['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'Cluster Labels'], axis = 1)
nycluster2 = newyork_merged.loc[newyork_merged['Cluster Labels'] == 1].drop(['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'Cluster Labels'], axis = 1)
nycluster3 = newyork_merged.loc[newyork_merged['Cluster Labels'] == 2].drop(['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'Cluster Labels'], axis = 1)
nycluster4 = newyork_merged.loc[newyork_merged['Cluster Labels'] == 3].drop(['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'Cluster Labels'], axis = 1)
nycluster5 = newyork_merged.loc[newyork_merged['Cluster Labels'] == 4].drop(['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'Cluster Labels'], axis = 1)

## Extract clusters in Toronto

In [56]:
trcluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0].drop(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude', 'Cluster Labels'], axis = 1)
trcluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1].drop(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude', 'Cluster Labels'], axis = 1)
trcluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2].drop(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude', 'Cluster Labels'], axis = 1)
trcluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3].drop(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude', 'Cluster Labels'], axis = 1)
trcluster5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4].drop(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude', 'Cluster Labels'], axis = 1)